In [ ]:
# dependecies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from scipy import stats
import tensorflow as tf
from tensorflow import keras
from keras import Input
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# for all my plots
palette = ['#E0B8AC', '#855648', '#6D312A', '#D0BEAA', '#CDA195', '#A49382', '#C7B7AB']

In [ ]:
data = pd.read_csv('personalFinanceDataset.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
# test; i just wanted to see what would happen
duplicatedRows = data[data.duplicated()]
duplicatedRows.head()

In [ ]:
data.shape